In [2]:
import pandas as pd

df = pd.read_excel('EIDOS_taxonomia.xlsx')[['nameid', 'name', 'taxonrank', 'taxonomicgroup']]

In [3]:
df = df[df['taxonrank'].isin(['Species', 'Subspecies']) & (df['nameid'] < 100000)]
df['chosen'] = False
df['words'] = df['name'].str.split(' ').apply(lambda x: [word.replace('.', '').replace(',', '').replace('(', '').replace(')', '').lower() for word in x if len(word) > 2])
df['all_words'] = df['words'].apply(lambda x: set(x))
len(df)

89306

In [4]:
df.sample(5, random_state=0)

,nameid,name,taxonrank,taxonomicgroup,chosen,words,all_words
11560,11655,Lonchura malacca,Species,Aves,False,"[lonchura, malacca]","{lonchura, malacca}"
8216,8224,Sedum rupestre L.,Species,Plantas vasculares,False,"[sedum, rupestre]","{sedum, rupestre}"
65383,69032,"Satyrus actaea (Esper, 1781)",Species,Invertebrados,False,"[satyrus, actaea, esper, 1781]","{1781, satyrus, actaea, esper}"
81290,85057,Oscillatoria curviceps,Species,Cromistas y Bacterias,False,"[oscillatoria, curviceps]","{curviceps, oscillatoria}"
89420,93232,Paropta paradoxa,Species,Invertebrados,False,"[paropta, paradoxa]","{paropta, paradoxa}"


In [10]:
MIN_LENGTH = 3
ONLY_FULL_WORDS = True

all_words = set()
all_words_dict = {}
for i, row in df.iterrows():
    if not isinstance(row['words'], list):
        continue
    words_to_add = [word for word in row['words'] if len(word) > MIN_LENGTH - 1 and isinstance(word, str)]
    all_words.update(words_to_add)
    if not ONLY_FULL_WORDS:
        to_add = []
        for word in words_to_add:
            to_add += [word[:length+1] for length in range(len(word))]
        all_words.update(to_add)
        all_words_dict[i] = set(to_add)
    else:
        all_words_dict[i] = set(words_to_add)
df['all_words'] = df.index.map(all_words_dict)
df['chosen_words'] = df['all_words']
all_words = sorted(all_words, key=lambda x: len(x), reverse=True) # Las palabras mas largas primero
word_counts = pd.Series(
    [word for words in df['all_words'] if isinstance(words, set) for word in words]
).value_counts()
len(all_words)

68666

In [12]:
try:
    words_to_exclude = set(pd.read_csv('palabras_para_filtrar.csv', header=None)[0])
except:
    words_to_exclude = set()
df['chosen'] = df['all_words'].apply(lambda words: isinstance(words, set) and bool(words & words_to_exclude))
df['chosen_words'] = df['all_words']
df.loc[df['chosen'], 'chosen_words'] = [set() for _ in range((df['chosen']).sum())]
word_counts_only_chosen = pd.Series(
    [word for words in df['chosen_words'] if isinstance(words, set) for word in words]
).value_counts()
words_df = pd.DataFrame({'all': word_counts, 'chosen': word_counts_only_chosen}).fillna(0)
words_df['chosen'] = words_df['chosen'].astype(int)
words_df[words_df['all'] < 1000].sort_values('chosen', ascending=False)[:20]

,all,chosen
fries,720,720
boiss,705,705
wollaston,700,700
1864,498,498
simon,497,497
canariensis,488,484
persoon,436,436
1908,392,392
navicula,386,386
1839,375,375


In [1]:
import pandas as pd
import os

# Celda para comprobar cuantas especies unicas hay descargadas en total
species_set = set()
for file in os.listdir('./IEPNB_filtrado'):
    sp = pd.read_excel(f'IEPNB_filtrado/{file}')
    species_set.update(sp['Código de la especie (id taxon)'])
len(species_set)

52783

In [ ]:
dfs = []
for file in os.listdir('./IEPNB_filtrado'):
    dfs.append(pd.read_excel(f'IEPNB_filtrado/{file}'))
full_df = pd.concat(dfs).sort_values(by='Código de la especie (id taxon)')
full_df.drop_duplicates(subset=['Código de la especie (id taxon)'], inplace=True)
full_df.to_excel('IEPNB_base_de_datos.xlsx', index=False)

with_descriptions = full_df[~full_df['Descripción'].isnull()]
with_descriptions.to_excel('IEPNB_descripciones.xlsx', index=False)
with_descriptions['Grupo taxonómico'].value_counts()

Grupo taxonómico
Plantas vasculares       770
Invertebrados            727
Mamíferos                132
Peces                     72
Plantas no vasculares     26
Reptiles                  18
Aves                      14
Algas                     10
Anfibios                   4
Cromistas y Bacterias      3
Hongos                     1
Name: count, dtype: int64

Tan solo 1777 tienen descripciones, apenas un 3,4% de las especies descargadas.